<a href="https://colab.research.google.com/github/hassaku12/manabiDX2025/blob/main/%E6%BC%94%E7%BF%9203%EF%BC%88%E8%A9%A6%E3%81%99%E7%94%A8%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データの読み取り

In [2]:
# まずはGoogleドライブにアクセスする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# japanize-matplotlibのインストール (実行環境に未導入の場合)
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 94.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=7707fa96c61a2f263da1e435419d89b95954246990d71e7b1ed4d4a7a59cec8b
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize-matplotlib


In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import japanize_matplotlib
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [6]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/マナビDX'

In [7]:
print("📊 データ読み込み中...")
train_df = pd.read_csv(base_dir + '/train.csv')
test_df = pd.read_csv(base_dir + '/test.csv')
submission_df = pd.read_csv(base_dir + '/sample_submit.csv', header=None)
assessment_df = pd.read_csv(base_dir + '/適合性判定シート一覧表.csv')

📊 データ読み込み中...


In [8]:
print("📊 追加データ読み込み中...")

# 約定データと商品リストを読み込み
try:
    trading_df = pd.read_csv(base_dir + '/約定データ一覧表.csv')
    product_df = pd.read_csv(base_dir + '/商品リスト.csv')

    print(f"✅ 約定データ: {trading_df.shape}")
    print(f"✅ 商品リスト: {product_df.shape}")
    print(f"✅ 約定データ列: {list(trading_df.columns)}")
    print(f"✅ 商品リスト列: {list(product_df.columns)}")

    HAS_ALL_DATA = True

except Exception as e:
    print(f"❌ データ読み込みエラー: {e}")
    HAS_ALL_DATA = False

📊 追加データ読み込み中...
✅ 約定データ: (37514, 20)
✅ 商品リスト: (786, 18)
✅ 約定データ列: ['取引日', '顧客ID', '取引コード', '商品名', '取得価額', '取得単価', '取引数量', '売却単価', '売却価額', '売却損益', '償還価額', '償還単価', '償還損益', '積立投資購入の新規/既存', '購入開始年月', '購入終了年月', 'オンライン取引フラグ', 'ゴール設定実施', 'ロスカット設定実施', 'ロスカット水準']
✅ 商品リスト列: ['商品名', '商品カテゴリ', 'リスクカテゴリ', '通貨', '勧誘留意商品', '解約手数料率', '発行日', '償還日', '利率', '発行日株価', '早期償還株価', 'ノックイン株価', '早期償還日', 'ノックイン日', '契約年月', '予定利率', 'ターゲット', 'ターゲット到達日']


## データ前処理

In [9]:
# 顧客属性データの前処理
assessment_df['取引日'] = pd.to_datetime(assessment_df['取引日'])
assessment_df_latest = assessment_df.sort_values(['顧客ID', '取引日']).drop_duplicates(subset='顧客ID', keep='last')
assessment_df_selected = assessment_df_latest[['顧客ID', '顧客年齢', '投資経験（株式）']]

In [10]:
# 顧客属性情報を結合
train_df = pd.merge(train_df, assessment_df_selected, on='顧客ID', how='left')
test_df = pd.merge(test_df, assessment_df_selected, on='顧客ID', how='left')

In [11]:
# 欠損値を補完
train_df['顧客年齢'] = train_df['顧客年齢'].fillna(train_df['顧客年齢'].mean())
test_df['顧客年齢'] = test_df['顧客年齢'].fillna(test_df['顧客年齢'].mean())
train_df['投資経験（株式）'] = train_df['投資経験（株式）'].fillna(0)
test_df['投資経験（株式）'] = test_df['投資経験（株式）'].fillna(0)

In [12]:
# 基準年月から年と月を抽出
train_df['year'] = train_df['基準年月'].str.split('-').str[0].astype(int)
train_df['month'] = train_df['基準年月'].str.split('-').str[1].astype(int)
test_df['year'] = test_df['基準年月'].str.split('-').str[0].astype(int)
test_df['month'] = test_df['基準年月'].str.split('-').str[1].astype(int)

## 特徴量エンジニアリング（すべてを関数で統一）

In [21]:
# 既存の関数群（必須）
def add_real_trading_features(df):
    """実際の約定データから特徴量作成"""
    try:
        trading_df['取引日'] = pd.to_datetime(trading_df['取引日'])
        past_trading = trading_df[trading_df['取引日'] <= '2021-11-30']

        customer_stats = past_trading.groupby('顧客ID').agg({
            '取得価額': ['count', 'sum', 'mean'],
            '売却損益': ['sum', 'count'],
            '償還損益': ['sum', 'count'],
            'オンライン取引フラグ': 'mean',
            '商品名': 'nunique',
            'ゴール設定実施': 'mean',
            'ロスカット設定実施': 'mean'
        }).fillna(0)

        customer_stats.columns = [
            '過去取引回数', '過去総取得額', '過去平均取得額',
            '売却損益合計', '売却回数', '償還損益合計', '償還回数',
            'デジタル利用率', '商品多様性', 'ゴール設定率', 'ロスカット設定率'
        ]

        customer_stats['過去累積損益'] = customer_stats['売却損益合計'] + customer_stats['償還損益合計']
        customer_stats['投資成功体験'] = (customer_stats['過去累積損益'] > 0).astype(int)
        customer_stats['平均利益率'] = customer_stats['過去累積損益'] / (customer_stats['過去総取得額'] + 1)
        customer_stats['リスク管理度'] = (customer_stats['ゴール設定率'] + customer_stats['ロスカット設定率']) / 2

        # 投資家タイプ分類
        customer_stats['投資家タイプ'] = 'その他'
        active_mask = (
            (customer_stats['過去取引回数'] > customer_stats['過去取引回数'].quantile(0.7)) &
            (customer_stats['投資成功体験'] == 1)
        )
        customer_stats.loc[active_mask, '投資家タイプ'] = 'アクティブ'

        careful_mask = customer_stats['リスク管理度'] > 0.5
        customer_stats.loc[careful_mask, '投資家タイプ'] = '慎重'

        digital_mask = customer_stats['デジタル利用率'] > 0.7
        customer_stats.loc[digital_mask, '投資家タイプ'] = 'デジタル'

        df = df.merge(customer_stats, left_on='顧客ID', right_index=True, how='left')

        numeric_cols = [col for col in customer_stats.columns if col != '投資家タイプ']
        for col in numeric_cols:
            df[col] = df[col].fillna(0)
        df['投資家タイプ'] = df['投資家タイプ'].fillna('その他')

        return df

    except Exception as e:
        print(f"取引特徴量エラー: {e}")
        return add_dummy_features(df)

In [22]:
def add_real_product_features(df):
    """実際の商品データから特徴量作成"""
    try:
        trading_with_product = trading_df.merge(
            product_df[['商品名', '商品カテゴリ', 'リスクカテゴリ', '通貨', '勧誘留意商品']],
            on='商品名', how='left'
        )

        customer_products = trading_with_product.groupby('顧客ID').agg({
            '商品カテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'その他',
            'リスクカテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Medium',
            '通貨': lambda x: (x == 'JPY').mean(),
            '勧誘留意商品': 'mean'
        })

        customer_products.columns = ['主要商品カテゴリ', '主要リスクカテゴリ', '円建て比率', '留意商品比率']

        risk_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
        customer_products['リスク指向度'] = customer_products['主要リスクカテゴリ'].map(risk_mapping).fillna(2)

        df = df.merge(customer_products, left_on='顧客ID', right_index=True, how='left')

        df['主要商品カテゴリ'] = df['主要商品カテゴリ'].fillna('その他')
        df['主要リスクカテゴリ'] = df['主要リスクカテゴリ'].fillna('Medium')
        df['円建て比率'] = df['円建て比率'].fillna(1.0)
        df['留意商品比率'] = df['留意商品比率'].fillna(0.0)
        df['リスク指向度'] = df['リスク指向度'].fillna(2)

        return df

    except Exception as e:
        print(f"商品特徴量エラー: {e}")
        return df

In [23]:
def add_dummy_features(df):
    """ダミー特徴量追加"""
    dummy_features = [
        '過去取引回数', '過去累積損益', '投資成功体験', 'デジタル利用率',
        '商品多様性', '主要リスクカテゴリ', '投資家タイプ', 'リスク管理度', 'リスク指向度'
    ]
    for feature in dummy_features:
        if feature not in ['主要リスクカテゴリ', '投資家タイプ']:
            df[feature] = 0
        else:
            df[feature] = 'その他' if feature == '投資家タイプ' else 'Medium'
    df['円建て比率'] = 1.0
    df['留意商品比率'] = 0.0
    return df

In [24]:
def create_complete_features(df):
    """完全版特徴量作成"""
    df = df.copy()

    # 基本特徴量
    df['資産規模'] = df['取得価額'] + df['時価価額']
    df['投資効率'] = df['時価価額'] / (df['取得価額'] + 1)
    df['年齢調整資産'] = df['時価価額'] / (df['顧客年齢'] + 1)
    df['顧客ランク'] = df['時価価額'].rank(pct=True)
    df['年齢×資産規模'] = df['顧客年齢'] * df['資産規模']
    df['含み損益率'] = np.where(df['取得価額'] != 0,
                                (df['時価価額'] - df['取得価額']) / df['取得価額'], 0)

    if HAS_ALL_DATA:
        df = add_real_trading_features(df)
        df = add_real_product_features(df)
    else:
        df = add_dummy_features(df)

    # フラグ特徴量
    df['高資産フラグ'] = (df['資産規模'] > df['資産規模'].quantile(0.8)).astype(int)
    df['含み損フラグ'] = (df['評価損益'] < 0).astype(int)
    df['シニアフラグ'] = (df['顧客年齢'] >= 65).astype(int)
    df['株式経験あり'] = (df['投資経験（株式）'] > 0).astype(int)

    return df

In [25]:
# 🚀 最終調整：新規特徴量追加
def create_final_features(df):
    """最後の+0.01を狙う特徴量"""
    df = df.copy()

    # 既存特徴量作成
    df = create_complete_features(df)

    # 【新規】高度な相互作用特徴量
    df['投資効率×年齢'] = df['投資効率'] * df['顧客年齢']
    df['リスク×経験'] = df['リスク指向度'] * df['投資経験（株式）']
    df['資産×取引頻度'] = df['資産規模'] * df['過去取引回数']

    # 【新規】比率特徴量
    df['成功率'] = df['投資成功体験'] / (df['過去取引回数'] + 1)
    df['効率ランク'] = df['投資効率'].rank(pct=True)
    df['年齢内資産順位'] = df.groupby('顧客年齢')['時価価額'].rank(pct=True)

    # 【新規】複合スコア
    df['総合投資力'] = (
        df['顧客ランク'] * 0.3 +
        df['効率ランク'] * 0.3 +
        df['投資成功体験'] * 0.2 +
        (df['過去取引回数'] / 50).clip(0, 1) * 0.2
    )

    # 【新規】異常値フラグ
    df['異常高資産'] = (df['時価価額'] > df['時価価額'].quantile(0.99)).astype(int)
    df['超アクティブ'] = (df['過去取引回数'] > df['過去取引回数'].quantile(0.95)).astype(int)

    return df

## 複数モデルのアンサンブル

In [26]:
# 🎯 最終モデル
class FinalEnsemble:
    def __init__(self):
        self.models = {
            'rf1': RandomForestClassifier(
                n_estimators=200, max_depth=12, min_samples_leaf=6,
                max_features=0.8, random_state=42, n_jobs=-1, class_weight='balanced'
            ),
            'et1': ExtraTreesClassifier(
                n_estimators=200, max_depth=14, min_samples_leaf=4,
                max_features=0.9, random_state=42, n_jobs=-1, class_weight='balanced'
            ),
            'rf2': RandomForestClassifier(
                n_estimators=150, max_depth=8, min_samples_leaf=12,
                max_features=0.6, random_state=123, n_jobs=-1, class_weight='balanced'
            )
        }

    def fit(self, X, y, term_id):
        print(f"\n🎯 Term {term_id} - 最終調整学習")
        print(f"データ: {X.shape}, 陽性率: {y.mean():.4f}")

        cv_scores = {}
        for name, model in self.models.items():
            model.fit(X, y)
            cv_score = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
            cv_scores[name] = cv_score.mean()
            print(f"  {name}: {cv_score.mean():.4f} (±{cv_score.std():.3f})")

        scores = np.array(list(cv_scores.values()))
        exp_scores = np.exp(scores * 3)
        self.weights = dict(zip(cv_scores.keys(), exp_scores / exp_scores.sum()))
        print(f"  最終重み: {self.weights}")

        return cv_scores

    def predict_proba(self, X):
        predictions = {}
        for name, model in self.models.items():
            predictions[name] = model.predict_proba(X)[:, 1]

        ensemble_pred = sum(
            self.weights[name] * predictions[name]
            for name in self.models.keys()
        )
        return ensemble_pred

# 🚀 最終パイプライン
def final_pipeline(train_df, test_df):
    """最終調整パイプライン"""
    print("🚀 最終調整パイプライン（0.65達成狙い）")
    print("="*60)

    # 最終特徴量作成
    train_final = create_final_features(train_df.copy())
    test_final = create_final_features(test_df.copy())

    # 最強特徴量セット
    final_features = [
        # 基本重要特徴量
        '時価価額', '顧客ランク', '年齢×資産規模', '過去取引回数',
        '資産規模', '投資効率', '年齢調整資産', '含み損益率',

        # 実データ特徴量
        '商品多様性', 'デジタル利用率', '投資成功体験', '過去累積損益',
        'リスク管理度', 'リスク指向度', '円建て比率',

        # 【新規】最終調整特徴量
        '投資効率×年齢', 'リスク×経験', '資産×取引頻度',
        '成功率', '効率ランク', '年齢内資産順位', '総合投資力',
        '異常高資産', '超アクティブ',

        # 基本情報
        '顧客年齢', '投資経験（株式）', 'year', '投資方針',

        # フラグ
        '高資産フラグ', '含み損フラグ', 'シニアフラグ', '株式経験あり'
    ]

    print(f"📊 最終特徴量: {len(final_features)}個")

    # Term別予測
    all_predictions = pd.DataFrame()
    term_aucs = []

    for term_id in range(1, 7):
        print(f"\n{'='*40}")
        print(f"Term {term_id} 最終調整")
        print(f"{'='*40}")

        # データ準備
        train_term = train_final[train_final[f'train_term_{term_id}'] == 1]
        test_term = test_final[test_final[f'test_term_{term_id}'] == 1]

        available_features = [f for f in final_features if f in train_term.columns]
        print(f"利用可能特徴量: {len(available_features)}個")

        X_train = train_term[available_features].fillna(0)
        y_train = train_term['y']
        X_test = test_term[available_features].fillna(0)

        # カテゴリ変数エンコーディング
        categorical_cols = ['投資方針', '投資家タイプ', '主要リスクカテゴリ']
        categorical_to_encode = [col for col in categorical_cols if col in X_train.columns]

        if categorical_to_encode:
            X_train = pd.get_dummies(X_train, columns=categorical_to_encode, drop_first=True, dtype=int)
            X_test = pd.get_dummies(X_test, columns=categorical_to_encode, drop_first=True, dtype=int)

            missing_cols = set(X_train.columns) - set(X_test.columns)
            for col in missing_cols:
                X_test[col] = 0
            X_test = X_test[X_train.columns]

        print(f"最終特徴量数: {X_train.shape[1]}個")

        # 最終モデル学習
        ensemble = FinalEnsemble()
        cv_scores = ensemble.fit(X_train, y_train, term_id)
        term_aucs.append(np.mean(list(cv_scores.values())))

        # 予測
        test_pred = ensemble.predict_proba(X_test)

        # 結果保存
        pred_df = pd.DataFrame({'ID': test_term['ID'], 'predict': test_pred})
        all_predictions = pd.concat([all_predictions, pred_df]) if not all_predictions.empty else pred_df

    # 最終結果
    avg_auc = np.mean(term_aucs)
    print(f"\n{'='*60}")
    print("🎯 最終調整結果")
    print(f"{'='*60}")

    for i, auc in enumerate(term_aucs, 1):
        print(f"Term {i}: {auc:.4f}")

    print(f"\n📈 最終CV-AUC: {avg_auc:.4f}")
    print(f"📊 予想パブリック: {avg_auc - 0.027:.4f}")

    if avg_auc >= 0.677:
        print("🎉 0.65達成可能性大！")
    elif avg_auc >= 0.67:
        print("✅ 0.64後半達成！")

    return all_predictions, avg_auc

## モデル評価の強化

In [18]:
## モデル評価関数
def evaluate_model_performance(y_true, y_pred_proba, threshold=0.23):
    """PoCで設定した指標で評価"""
    y_pred = (y_pred_proba >= threshold).astype(int)

    metrics = {
        'AUC': roc_auc_score(y_true, y_pred_proba),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0)
    }

    print(f"AUC: {metrics['AUC']:.4f} (目標: 0.7以上)")
    print(f"Precision: {metrics['Precision']:.4f} (目標: 0.4以上)")
    print(f"Recall: {metrics['Recall']:.4f} (目標: 0.6以上)")
    print(f"F1 Score: {metrics['F1']:.4f} (目標: 0.5以上)")

    return metrics

## メイン処理（改善版）

In [27]:
print("🚀 最終調整実行（0.65達成狙い）")
final_predictions, final_auc = final_pipeline(train_df, test_df)

🚀 最終調整実行（0.65達成狙い）
🚀 最終調整パイプライン（0.65達成狙い）
📊 最終特徴量: 32個

Term 1 最終調整
利用可能特徴量: 32個
最終特徴量数: 33個

🎯 Term 1 - 最終調整学習
データ: (80000, 33), 陽性率: 0.1972
  rf1: 0.6689 (±0.019)
  et1: 0.6676 (±0.016)
  rf2: 0.6849 (±0.014)
  最終重み: {'rf1': np.float64(0.3283445751389319), 'et1': np.float64(0.3271140039404394), 'rf2': np.float64(0.3445414209206287)}

Term 2 最終調整
利用可能特徴量: 32個
最終特徴量数: 33個

🎯 Term 2 - 最終調整学習
データ: (82000, 33), 陽性率: 0.1976
  rf1: 0.6680 (±0.019)
  et1: 0.6667 (±0.015)
  rf2: 0.6831 (±0.016)
  最終重み: {'rf1': np.float64(0.3286809601766906), 'et1': np.float64(0.32739025581951753), 'rf2': np.float64(0.34392878400379195)}

Term 3 最終調整
利用可能特徴量: 32個
最終特徴量数: 33個

🎯 Term 3 - 最終調整学習
データ: (84000, 33), 陽性率: 0.1980
  rf1: 0.6613 (±0.019)
  et1: 0.6668 (±0.016)
  rf2: 0.6761 (±0.018)
  最終重み: {'rf1': np.float64(0.32655385997904557), 'et1': np.float64(0.3320207911316294), 'rf2': np.float64(0.34142534888932496)}

Term 4 最終調整
利用可能特徴量: 32個
最終特徴量数: 33個

🎯 Term 4 - 最終調整学習
データ: (86000, 33), 陽性率: 0.1985
  rf1: 0

In [29]:
# 提出ファイル作成
submission_df[1] = final_predictions['predict']
submission_df.to_csv(base_dir + '/final_submission.csv', index=False, header=False)

print(f"\n🎯 最終調整完了！")
print(f"CV-AUC: {final_auc:.4f}")
print(f"予想改善: 0.6381 → {final_auc - 0.027:.4f}")
print("提出ファイル: final_submission.csv")


🎯 最終調整完了！
CV-AUC: 0.6694
予想改善: 0.6381 → 0.6424
提出ファイル: final_submission.csv
